In [1]:
from colorsys import rgb_to_hsv
import pandas as pd

In [2]:
#clean data
color_df = pd.read_csv('lipstick_colors.csv')
# color_df.drop(color_df['Unnamed: 0'])
color_df = color_df.drop(['Unnamed: 0'],axis = 1)
color_df = color_df.fillna(value=str(0))

In [34]:
# from colorsys import rgb_to_hsv
# import pandas as pd

# import re 

# #Clean Dataset
# pattern = r'(\d+)'
# def get_tuple(row):
#     match = re.findall(pattern, row)
#     tuple_=(float(match[0]), float(match[1]), float(match[2]))
#     return tuple_
# color_df['(r, g, b)'] = color_df['(r, g, b)'].apply(get_tuple)

# pattern_ = r'^\w{3} \w{2} \w{5}: '
# def remove_nostock(row):
# #     pattern = r'^\w{3} \w{2} \w{5}: '
#     match = re.search(pattern_, row)
#     if match:
#         row = row[14:]
# #     row=row
#     return row
# color_df['color'] = color_df['color'].apply(remove_nostock)

# add a column to convert rgb to hsv
# def to_hsv( color ): 
#     """ converts color tuples to floats and then to hsv """
#     return rgb_to_hsv(*[x/255.0 for x in color]) #rgb_to_hsv wants floats!
# color_df['hsv'] = color_df['(r, g, b)'].apply(to_hsv)

close_color ={}
l=[]
def min_color_diff( color_to_match):
    """ returns the `(distance, color_name)` with the minimal distance to `colors` choose 5 closest color"""
    from colorsys import rgb_to_hsv
    import pandas as pd

    import re 
    
    #clean data
    color_df = pd.read_csv('lipstick_colors.csv')
    # color_df.drop(color_df['Unnamed: 0'])
    color_df = color_df.drop(['Unnamed: 0'],axis = 1)
    color_df = color_df.fillna(value=str(0))
    
    pattern = r'(\d+)'
    def get_tuple(row):
        match = re.findall(pattern, row)
        tuple_=(float(match[0]), float(match[1]), float(match[2]))
        return tuple_
    color_df['(r, g, b)'] = color_df['(r, g, b)'].apply(get_tuple)

    pattern_ = r'^\w{3} \w{2} \w{5}: '
    def remove_nostock(row):
#     pattern = r'^\w{3} \w{2} \w{5}: '
        match = re.search(pattern_, row)
        if match:
            row = row[14:]
#     row=row
        return row
    color_df['color'] = color_df['color'].apply(remove_nostock)

    def to_hsv( color ): 
        """ converts color tuples to floats and then to hsv """
        return rgb_to_hsv(*[x/255.0 for x in color]) #rgb_to_hsv wants floats!
    color_df['hsv'] = color_df['(r, g, b)'].apply(to_hsv)

    def color_dist( c1, c2):
        """ returns the squared euklidian distance between two color vectors in hsv space """
        return sum( (a-b)**2 for a,b in zip(to_hsv(c1),to_hsv(c2)) )

    for i in range(len(color_df['hsv'])):
        close_color[color_df['hsv'][i]] = (color_df['product_name'][i], color_df['color'][i], color_dist(color_to_match, color_df['hsv'][i]))
    close_color_ = sorted(close_color.items(), key=lambda x: x[1][1])
    for key, value in close_color_:
        if value[0] or value[1]  != None:
            l.append([value[0], value[1]])
        if value[1] == None:
            l.append(value[0])
        if value[0] ==None:
            l.append(value[1])
    return l[:5]

In [35]:
color_to_match = (255,255,0)
z = min_color_diff( color_to_match)
print(min_color_diff( color_to_match))

[['On The Dot Mini Le Marc Liquid Lip Crayon Trio', ''], ['Rouge Shine Lipstick', ' 19 v.i.p'], ['Rouge Shine Lipstick', ' 42 walk of fame'], ['Rouge Cream Lipstick', ' Magnetism 27 deep browned bronze'], ['Sexy Nude Gloss Balm Kit', '0']]
